# Necessary Packages and Imports

In [16]:
!pip install -q docplex==2.27.239
!pip install -q cplex==22.1.1.2

In [17]:
import math
import time
import random
import docplex.mp

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from docplex.mp.model import Model
from cplex import Cplex
from collections import deque
from collections import defaultdict
from google.colab import drive

# Helper Classes

In [18]:
class Shipment:
    def __init__(self, start, end, volume, has_route=False, route=None):
      self.start = start
      self.end = end
      self.volume = volume
      self.has_route = has_route
      self.route = route

    def __eq__(self, other):
      if isinstance(other, Shipment):
        return (self.start == other.start and
               self.end == other.end and
               self.volume == other.volume and
               self.has_route == other.has_route and
               self.route == other.route)
      return False

    def toString(self):
      return "from " + str(self.start) + " to " + str(self.end) + " carrying " + str(self.volume)

    def __hash__(self):
      return hash((self.start, self.end, self.volume, self.has_route, self.route))

    def get_source(self):
      return self.start

    def get_sink(self):
      return self.end

    def get_vol(self):
      return self.volume

    def assign_route(self, route):
      self.route = route
      self.has_route = True

    def get_route(self):
      if self.has_route == False:
        return None
      else:
        return self.route

In [19]:
class Graph:
  def __init__(self, shipments=None):
    self.graph = defaultdict(dict)  # Adjacency list with weights
    self.shipments = shipments

  def add_edge(self, u, v, weight):
    self.graph[u][v] = weight
    self.graph[v][u] = weight

  def draw_graph(self):
    G = nx.DiGraph()  # Create a directed graph

    # Add nodes and edges with weights as attributes
    for node, neighbors in self.graph.items():
        G.add_node(node)
        for neighbor, weight in neighbors.items():
            G.add_edge(node, neighbor, weight=weight)

    pos = nx.spring_layout(G)  # Use spring layout algorithm

    # Customize node and edge appearance
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=12)
    nx.draw_networkx_edges(G, pos, width=2, alpha=0.7)
    nx.draw_networkx_labels(G, pos, font_size=12, font_color='black')  # Add this line to label nodes

    plt.axis('off')
    plt.show()

  def floyd_warshall(self):
    nodes = list(self.graph.keys())
    num_nodes = len(nodes)
    self.distances = {node: {node: float('inf') for node in nodes} for node in nodes}
    self.paths = {node: {node: None for node in nodes} for node in nodes}

    for node in nodes:
        self.distances[node][node] = 0

    for u in self.graph:
        for v in self.graph[u]:
            self.distances[u][v] = self.graph[u][v]
            self.paths[u][v] = u

    count = 0
    total = len(nodes)**3
    for k in nodes:
        for i in nodes:
            for j in nodes:
                if self.distances[i][j] > self.distances[i][k] + self.distances[k][j]:
                   self.distances[i][j] = self.distances[i][k] + self.distances[k][j]
                   self.paths[i][j] = self.paths[k][j]
                progress = (count + 1) / total * 100
                count += 1
        print(f"\rProgress: {progress:.2f}%", end="")


    return self.distances, self.paths

# Helper Functions

In [20]:
def get_path(paths_dict, start, end):
  if paths_dict[start][end] is None:
     return []

  path = []
  curr = end

  while curr != start:
     pred = paths_dict[start][curr]
     path.append((pred, curr))
     curr = pred

  # path.append(start)
  path.reverse()
  return path

In [21]:
def find_all_paths(s, G, visited=[], path=[], show=False):
  source = s.get_source()
  sink = s.get_sink()

  all_paths = find_all_paths_helper(source, sink, visited, path, [], G, show=show)

  all_paths_edges = []
  for path in all_paths:
    path_edges = []
    for j in range(0, len(path)-1):
      path_edges.append((path[j], path[j+1]))
    all_paths_edges.append(path_edges)

  return all_paths_edges

def find_all_paths_helper(source, sink, visited, path, all_paths, G, max_paths_num=5, show=False, start_time=None, max_time=2):
     # Start the timer if it's not already started
    if start_time is None:
        start_time = time.time()

    if len(all_paths) >= max_paths_num or (time.time() - start_time) > max_time:
        return all_paths

    visited.append(source)
    path.append(source)

    if source == sink:
        all_paths.append(path.copy())
        if show == True:
          print(path)
        if len(all_paths) >= max_paths_num:
            return all_paths
    else:
        for neighbor, weight in G.graph[source].items():
            if neighbor not in visited:
                find_all_paths_helper(neighbor, sink, visited, path, all_paths, G, max_paths_num, show=show, start_time=start_time, max_time=max_time)

    visited.remove(source)
    path.remove(source)

    return all_paths

In [22]:
def bfs(start_node, subgraph, visited):
    queue = deque([start_node])
    connected_component = []

    while queue:
        node = queue.popleft()
        if not visited[node]:
            visited[node] = True
            connected_component.append(node)
            for neighbor in subgraph[node]:
                if not visited[neighbor]:
                    queue.append(neighbor)

    return connected_component

def find_all_connected_components(subgraph):
    visited = {node: False for node in subgraph}
    connected_components = []

    for node in subgraph:
        if not visited[node]:
            component = bfs(node, subgraph, visited)
            connected_components.append(component)

    return connected_components

In [23]:
def R(e, s, G):
  cand_paths = shipments_routes_dict[s]
  e_paths = []
  for path in cand_paths:
    if (e[0], e[1]) in path:
      e_paths.append(path)
  return e_paths

In [24]:
def R_s(e, G):
  all_cand_paths = []
  for s in shipments:
    for path in shipments_routes_dict[s]:
      all_cand_paths.append(path)
  e_paths = []
  for path in all_cand_paths:
    if (e[0], e[1]) in path:
      e_paths.append(path)
  return e_paths

In [25]:
def scheduled(e, s, G):
  if len(R(e, s, G)) > 0:
    return True
  else:
    return False

In [26]:
def d(e):
  (_, _, weight) = e
  return weight

In [27]:
def w(path):
  sum = 0
  for e in path:
    sum += edge_weight_dict[e]
  return sum

In [28]:
def v(s):
  return s.get_vol()

In [29]:
def l(s):
  load = Shipment.get_vol(s)
  return load

# Reading Input File

In [ ]:
drive.mount('/content/drive')

In [ ]:
file_path = "PATH_TO_DRIVE/Shipment Rerouting Algorithms/Networks/Braess_net.tntp"

# Reading the file
with open(file_path, 'r') as file:
    content = file.read()

In [ ]:
graph_data = pd.read_csv(file_path, sep='\t', nrows=3)
num_nodes = int(graph_data.loc[0][0][18:])
num_edges = int(graph_data.loc[2][0][18:])

net = pd.read_csv(file_path, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)

df = net

# Generate Problem Instance

In [ ]:
# Example usage
# set of terminals on G
V = [str(i+1) for i in range(0, num_nodes)]

# set of edges on G (each edge e = (start terminal, ending terminal, weight))
E = []
for index, row in df.iterrows():
  E.append((str(int(row['init_node'])), str(int(row['term_node'])), row['length']))

unweighted_edges = []
for e in E:
  unweighted_edges.append((e[0], e[1]))
  unweighted_edges.append((e[1], e[0]))

weights = []
for e in E:
  weights.append(e[2])
  weights.append(e[2])

edge_weight_dict = dict(zip(unweighted_edges, weights))

# num of trucks per edge
T = [1 for e in unweighted_edges]

# num_truck dict (takes in edge and returns number of trucks on edge)
edge_trucks_dict = dict(zip(unweighted_edges, T))
truck_capacity = 101

G = Graph()

for start, end, weight in E:
  G.add_edge(start, end, weight)

all_edge_truck_dict = dict(zip(E, T))
def t_max(e):
  return all_edge_truck_dict[e]

c_vol = truck_capacity

# G.draw_graph()

# Preprocessing Distances

In [ ]:
floyd_warshall_dists, floyd_warshall_paths = G.floyd_warshall()

# Generating Shipments

In [ ]:
# shipments can be provided manually or generated at random

# Manually adding shipments
shipments = [Shipment("1", "4", 50), Shipment("2", "3", 50), Shipment("1", "3", 50), Shipment("2", "4", 50)]

shipments_routes = []
for s in shipments:
    paths = find_all_paths(s, G, [], [])
    shipments_routes.append(paths)

In [ ]:
# Automated generation of shipments
# num_shipments =
# shipments = []
# shipments_routes = []
# num_shipments_generated = 0
# total = num_shipments
# while len(shipments) < num_shipments:
#   start = str(random.randint(1, num_nodes))
#   end = str(random.randint(1, num_nodes))
#   while (end == start):
#     end = str(random.randint(1, num_nodes))
#   s = Shipment(start, end, 50)
#   # print(s.toString())
#   paths = find_all_paths(s, G, [], [], show=False)
#   if len(paths) > 0:
#     progress = (num_shipments_generated + 1) / total * 100
#     num_shipments_generated += 1
#     shipments.append(s)
#     shipments_routes.append(paths)
#     print(f"\rProgress: {progress:.2f}%", end="")

In [ ]:
shipments_routes_dict = dict(zip(shipments, shipments_routes))

# Obtaining minimum paths for each shipment

In [ ]:
shipments_min_paths = []
for s in shipments:
  all_paths = shipments_routes_dict[s]
  if all_paths == []:
    raise ValueError("No paths found")
  min_path = all_paths[0]
  for path in all_paths:
    if w(path) < w(min_path):
      min_path = path
  shipments_min_paths.append(min_path)

In [ ]:
shipment_min_path_dict = dict(zip(shipments, shipments_min_paths))
min_path_shipment_dict = dict(zip([tuple(path) for path in shipments_min_paths], shipments))

# Merge Run Function

In [ ]:
def merge(R1, R2, min_path_shipment_dict):
  s1 = min_path_shipment_dict[tuple(R1)]
  s2 = min_path_shipment_dict[tuple(R2)]
  l1 = l(s1)
  l2 = l(s2)

  # print(R1)
  s1 = R1[0][0]
  t1 = R1[(len(R1)-1)][1]

  # print(R2)
  s2 = R2[0][0]
  t2 = R2[(len(R2)-1)][1]

  merged_path_cost = float('infinity')
  merged_path = []

  if truck_capacity < max(l1, l2):
    raise ValueError("Impossible to Transport")

  dist_t1_s2 = floyd_warshall_dists[t1][s2]
  path_t1_s2 = get_path(floyd_warshall_paths, t1, s2)

  dist_t2_s1 = floyd_warshall_dists[t2][s1]
  path_t2_s1 = get_path(floyd_warshall_paths, t2, s1)

  dist_s1_s2 = floyd_warshall_dists[s1][s2]
  path_s1_s2 = get_path(floyd_warshall_paths, s1, s2)

  dist_t1_t2 = floyd_warshall_dists[t1][t2]
  path_t1_t2 = get_path(floyd_warshall_paths, t1, t2)

  if truck_capacity < (l1 + l2):
    if dist_t1_s2 < dist_t2_s1:
      merged_path.append(R1)
      merged_path.append(path_t1_s2)
      merged_path.append(R2)
      merged_path = [item for sublist in merged_path for item in sublist]
      merged_path_cost = w(R1) + w(R2) + dist_t1_s2

      combined_shipment = Shipment(merged_path[0][0], merged_path[(len(merged_path)-1)][1], max(l1, l2))

      return combined_shipment, merged_path, merged_path_cost
    else:
      merged_path.append(R2)
      merged_path.append(path_t2_s1)
      merged_path.append(R1)
      merged_path = [item for sublist in merged_path for item in sublist]
      merged_path_cost = w(R1) + w(R2) + dist_t2_s1

      combined_shipment = Shipment(merged_path[0][0], merged_path[(len(merged_path)-1)][1], max(l1, l2))

      return combined_shipment, merged_path, merged_path_cost
  else:
    dist_s1_t1 = floyd_warshall_dists[s1][t1]
    path_s1_t1 = get_path(floyd_warshall_paths, s1, t1)

    dist_s1_t2 = floyd_warshall_dists[s1][t2]
    path_s1_t2 = get_path(floyd_warshall_paths, s1, t2)

    dist_s2_t1 = floyd_warshall_dists[s2][t1]
    path_s2_t1 = get_path(floyd_warshall_paths, s2, t1)

    dist_s2_t2 = floyd_warshall_dists[s2][t2]
    path_s2_t2 = get_path(floyd_warshall_paths, s2, t2)

    path_t2_t1 = get_path(floyd_warshall_paths, t2, t1)
    dist_t2_t1 = dist_t1_t2

    path_s2_s1 = get_path(floyd_warshall_paths, s2, s1)
    dist_s2_s1 = dist_s1_s2

    R_s1_t1_s2_t2 = []
    R_s1_t1_s2_t2.extend(path_s1_t1)
    R_s1_t1_s2_t2.extend(path_t1_s2)
    R_s1_t1_s2_t2.extend(path_s2_t2)
    dist_s1_t1_s2_t2 = dist_s1_t1 + dist_t1_s2 + dist_s2_t2

    R_s1_s2_t1_t2 = []
    R_s1_s2_t1_t2.extend(path_s1_s2)
    R_s1_s2_t1_t2.extend(path_s2_t1)
    R_s1_s2_t1_t2.extend(path_t1_t2)
    dist_s1_s2_t1_t2 = dist_s1_s2 + dist_s2_t1 + dist_t1_t2

    R_s1_s2_t2_t1 = []
    R_s1_s2_t2_t1.extend(path_s1_s2)
    R_s1_s2_t2_t1.extend(path_s2_t2)
    R_s1_s2_t2_t1.extend(path_t2_t1)
    dist_s1_s2_t2_t1 = dist_s1_s2 + dist_s2_t2 + dist_t2_t1

    R_s2_t2_s1_t1 = []
    R_s2_t2_s1_t1.extend(path_s2_t2)
    R_s2_t2_s1_t1.extend(path_t2_s1)
    R_s2_t2_s1_t1.extend(path_s1_t1)
    dist_s2_t2_s1_t1 = dist_s2_t2 + dist_t2_s1 + dist_s1_t1

    R_s2_s1_t2_t1 = []
    R_s2_s1_t2_t1.extend(path_s2_s1)
    R_s2_s1_t2_t1.extend(path_s1_t2)
    R_s2_s1_t2_t1.extend(path_t2_t1)
    dist_s2_s1_t2_t1 = dist_s2_s1 + dist_s1_t2 + dist_t2_t1

    R_s2_s1_t1_t2 = []
    R_s2_s1_t1_t2.extend(path_s2_s1)
    R_s2_s1_t1_t2.extend(path_s1_t1)
    R_s2_s1_t1_t2.extend(path_t1_t2)
    dist_s2_s1_t1_t2 = dist_s2_s1 + dist_s1_t1 + dist_t1_t2

    route_dist_dict = {
      dist_s1_t1_s2_t2: R_s1_t1_s2_t2,
      dist_s1_s2_t1_t2: R_s1_s2_t1_t2,
      dist_s1_s2_t2_t1: R_s1_s2_t2_t1,
      dist_s2_t2_s1_t1: R_s2_t2_s1_t1,
      dist_s2_s1_t2_t1: R_s2_s1_t2_t1,
      dist_s2_s1_t1_t2: R_s2_s1_t1_t2
    }

    min_route_dist = float('infinity')
    min_route = None
    for dist, route in route_dist_dict.items():
      if dist < min_route_dist:
        min_route_dist = dist
        min_route = route

    combined_shipment = Shipment(min_route[0][0], min_route[(len(min_route)-1)][1], max(l1, l2))

    return combined_shipment, min_route, min_route_dist

# Marginal Cost Function

In [ ]:
# fix marginal cost to only use paths
def marginal_cost(R1, R2, min_path_shipment_dict):
  s1 = min_path_shipment_dict[tuple(R1)]
  s2 = min_path_shipment_dict[tuple(R2)]
  l1 = l(s1)
  l2 = l(s2)

  s1 = R1[0][0]
  t1 = R1[(len(R1)-1)][1]

  s2 = R2[0][0]
  t2 = R2[(len(R2)-1)][1]

  dist_s2_t1 = floyd_warshall_dists[s2][t1]
  dist_t1_s2 = floyd_warshall_dists[t1][s2]

  dist_s1_t2 = floyd_warshall_dists[s1][t2]
  dist_t2_s1 = floyd_warshall_dists[t2][s1]

  dist_s1_s2 = floyd_warshall_dists[s1][s2]
  dist_s2_s1 = floyd_warshall_dists[s2][s1]

  dist_t1_t2 = floyd_warshall_dists[t1][t2]
  dist_t2_t1 = floyd_warshall_dists[t2][t1]

  dist_s1_t1 = floyd_warshall_dists[s1][t1]
  dist_s2_t2 = floyd_warshall_dists[s2][t2]

  paths_dists = [dist_s1_t1 + dist_t1_s2 + dist_s2_t2, dist_s1_s2 + dist_s2_t1 + dist_t1_t2, dist_s1_s2 + dist_s2_t2 + dist_t2_t1, dist_s2_t2 + dist_t2_s1 + dist_s1_t1, dist_s2_s1 + dist_s1_t2 + dist_t2_t1, dist_s2_s1 + dist_s1_t1 + dist_t1_t2]

  if truck_capacity < l1 + l2:
    return (w(R1) + w(R2) + min(dist_t1_s2, dist_t2_s1)) - (w(R1) + w(R2)), False
  else:
    min_dist = 10**9

    for p in paths_dists:
        dist = p
        if dist < min_dist:
            min_dist = dist

    twisted = True
    marge_cost = min_dist - w(R1) - w(R2)

    if min_dist == dist_s1_t1 + dist_t1_s2 + dist_s2_t2 or min_dist == dist_s2_t2 + dist_t2_s1 + dist_s1_t1:
        twisted = False
        return marge_cost, twisted

    return marge_cost, twisted

# Classical Algorithms

In [ ]:
def IP(shipments, K):
    model = Model(name='IP Formulation', log_output=True)

    t = model.continuous_var_dict(keys=E, lb={e: 0 for e in E}, ub=t_max, name=lambda e: "{}_t".format(e))

    # Objective: Minimize the total truck distance
    # Define and set the objective function
    objective = model.sum(d(e) * t[e] for e in E)
    model.minimize(objective)

    # setting up binary y_rs
    y_r_arr = {}

    for s in shipments:
        routes = shipments_routes_dict[s]
        y_r_s = model.binary_var_dict(keys=[tuple(r) for r in routes], name=lambda r: f"y_{s}_{r}")
        y_r_arr[s] = y_r_s

    # Route-shipment constraints: For each shipment s, exactly one associated candidate route is used
    for s in shipments:
        y_r = y_r_arr[s]
        model.add_constraint(model.sum(y_r[tuple(r)] for r in y_r.keys()) == 1)

    # Capacity constraints
    for e in E:
        for s in shipments:
            if scheduled(e, s, G):
                routes_scheduled_on_e = R(e, s, G)
                y_r = y_r_arr[s]
                model.add_constraint(
                model.sum(v(s) * y_r[tuple(r)] for r in routes_scheduled_on_e) <= (c_vol * t[e])
            )

    model.parameters.benders.strategy = 3

    # Optimize the model
    solution = model.solve()
    # Get the number of variables
    num_vars = model.number_of_variables
    # print(f"The number of variables in the model: {num_vars}")

    # Get the number of constraints
    num_constraints = model.number_of_constraints
    # print(f"The number of constraints in the model: {num_constraints}")

    # shipment_routes = []
    # for s in shipments:
    #     for r in y_r_arr[s].keys():
    #         if y_r_arr[s][r].solution_value == 1.0:
    #             shipment_routes.append(list(r))

    # shipment_min_path_dict = dict(zip(shipments, shipment_routes))
    # min_path_shipment_dict = dict(zip([tuple(path) for path in shipment_routes], shipments))

    # _, routes = WM_helper(shipments, shipment_routes, K, shipment_min_path_dict, min_path_shipment_dict)

    return routes

In [ ]:
def WM(shipments, K):
  shipment_min_path_dict_copy = shipment_min_path_dict.copy()
  min_path_shipment_dict_copy = min_path_shipment_dict.copy()

  shipment_routes = [shipment_min_path_dict_copy[s] for s in shipments] # make this a local copy and mutate it
  index_shipment_routes = [i for i in range(0, len(shipment_routes))]
  R = dict(zip(index_shipment_routes, shipment_routes))

  while len(shipment_routes) > K:
    if len(shipment_routes) % 2 == 0:
        index_shipment_routes = [i for i in range(0, len(shipment_routes))]
        R = dict(zip(index_shipment_routes, shipment_routes))
        # print("R:")
        # print(R)

        route_pairs = [(i, j) for j in range(1, len(index_shipment_routes)) for i in range(0, j)]
        # print("route pairs:")
        # print(route_pairs)

        model = Model(name='min cost matching')
        x = model.binary_var_dict(route_pairs, name='x')

        objective = model.sum(marginal_cost(R[a], R[b], min_path_shipment_dict_copy)[0] * x[(a, b)] for (a, b) in route_pairs)
        model.minimize(objective)

        for k in range(len(shipment_routes)):
          model.add_constraint(model.sum(x[i, j] for (i, j) in route_pairs if i == k or j == k) == 1)

        # Optimize the model
        solution = model.solve()

        new_routes = []

        selected_pairs = [(i, j) for (i, j) in route_pairs if x[(i, j)].solution_value == 1]
        # print("Selected Route Pairs:", selected_pairs)
        # print("Routes Before")
        # print(shipment_routes)
        for pair in selected_pairs:
          s1 = pair[0]
          s2 = pair[1]
          R1 = R[s1]
          # print("Route 1")
          # print(R1[0:5])
          R2 = R[s2]
          # print("Route 2")
          # print(R2[0:5])
          combined_shipment, merged_path, cost = merge(R1, R2, min_path_shipment_dict_copy)
          # print("Merged Path")
          # print(merged_path[0:5])
          new_routes.append(merged_path)
          # print("Routes After")
          # print(new_routes)
          shipment_min_path_dict_copy[combined_shipment] = merged_path
          min_path_shipment_dict_copy[tuple(merged_path)] = combined_shipment
    else:
        excluded_index = len(shipment_routes) - 1
        excluded_path = R[excluded_index]
        index_shipment_routes = [i for i in range(0, excluded_index)]
        R = dict(zip(index_shipment_routes, shipment_routes))
        # print("R:")
        # print(R)

        route_pairs = [(i, j) for j in range(1, len(index_shipment_routes)) for i in range(0, j)]
        # print("route pairs:")
        # print(route_pairs)

        model = Model(name='min cost matching')
        x = model.binary_var_dict(route_pairs, name='x')

        objective = model.sum(marginal_cost(R[a], R[b], min_path_shipment_dict_copy)[0] * x[(a, b)] for (a, b) in route_pairs)
        model.minimize(objective)

        for k in range(len(index_shipment_routes)):
          model.add_constraint(model.sum(x[i, j] for (i, j) in route_pairs if i == k or j == k) == 1)

        # Optimize the model
        solution = model.solve()

        new_routes = []
        new_routes.append(excluded_path)

        selected_pairs = [(i, j) for (i, j) in route_pairs if x[(i, j)].solution_value == 1]
        # print("Selected Route Pairs:", selected_pairs)
        # print("Routes Before")
        # print(shipment_routes)
        for pair in selected_pairs:
          s1 = pair[0]
          s2 = pair[1]
          R1 = R[s1]
          # print("Route 1")
          # print(R1)
          R2 = R[s2]
          # print("Route 2")
          # print(R2)
          combined_shipment, merged_path, cost = merge(R1, R2, min_path_shipment_dict_copy)
          # print("Merged Path")
          # print(merged_path)
          new_routes.append(merged_path)
          # print("Routes After")
          # print(new_routes)
          shipment_min_path_dict_copy[combined_shipment] = merged_path
          min_path_shipment_dict_copy[tuple(merged_path)] = combined_shipment

    shipment_routes = new_routes.copy()

  return shipment_routes

In [ ]:
def WM_helper(shipments, shipment_routes, K, passed_shipment_min_path_dict, passed_min_path_shipment_dict):
  shipment_min_path_dict_copy = passed_shipment_min_path_dict.copy()
  min_path_shipment_dict_copy = passed_min_path_shipment_dict.copy()

  index_shipment_routes = [i for i in range(0, len(shipment_routes))]
  R = dict(zip(index_shipment_routes, shipment_routes))

  while len(shipment_routes) > K:
    if len(shipment_routes) % 2 == 0:
        index_shipment_routes = [i for i in range(0, len(shipment_routes))]
        R = dict(zip(index_shipment_routes, shipment_routes))
        # print("R:")
        # print(R)

        route_pairs = [(i, j) for j in range(1, len(index_shipment_routes)) for i in range(0, j)]
        # print("route pairs:")
        # print(route_pairs)

        model = Model(name='min cost matching')
        x = model.binary_var_dict(route_pairs, name='x')

        objective = model.sum(marginal_cost(R[a], R[b], min_path_shipment_dict_copy)[0] * x[(a, b)] for (a, b) in route_pairs)
        model.minimize(objective)

        for k in range(len(shipment_routes)):
          model.add_constraint(model.sum(x[i, j] for (i, j) in route_pairs if i == k or j == k) == 1)

        # Optimize the model
        solution = model.solve()

        new_routes = []
        new_shipments = []

        selected_pairs = [(i, j) for (i, j) in route_pairs if x[(i, j)].solution_value == 1]
        # print("Selected Route Pairs:", selected_pairs)
        # print("Routes Before")
        # print(shipment_routes)
        for pair in selected_pairs:
          s1 = pair[0]
          s2 = pair[1]
          R1 = R[s1]
          # print("Route 1")
          # print(R1[0:5])
          R2 = R[s2]
          # print("Route 2")
          # print(R2[0:5])
          combined_shipment, merged_path, cost = merge(R1, R2, min_path_shipment_dict_copy)
          # print("Merged Path")
          # print(merged_path[0:5])
          new_routes.append(merged_path)
          new_shipments.append(combined_shipment)
          # print("Routes After")
          # print(new_routes)
          shipment_min_path_dict_copy[combined_shipment] = merged_path
          min_path_shipment_dict_copy[tuple(merged_path)] = combined_shipment
    else:
        excluded_index = len(shipment_routes) - 1
        excluded_shipment = shipments[excluded_index]
        excluded_path = R[excluded_index]
        index_shipment_routes = [i for i in range(0, excluded_index)]
        R = dict(zip(index_shipment_routes, shipment_routes))
        # print("R:")
        # print(R)

        route_pairs = [(i, j) for j in range(1, len(index_shipment_routes)) for i in range(0, j)]
        # print("route pairs:")
        # print(route_pairs)

        model = Model(name='min cost matching')
        x = model.binary_var_dict(route_pairs, name='x')

        objective = model.sum(marginal_cost(R[a], R[b], min_path_shipment_dict_copy)[0] * x[(a, b)] for (a, b) in route_pairs)
        model.minimize(objective)

        for k in range(len(index_shipment_routes)):
          model.add_constraint(model.sum(x[i, j] for (i, j) in route_pairs if i == k or j == k) == 1)

        # Optimize the model
        solution = model.solve()

        new_routes = []
        new_shipments = []
        new_routes.append(excluded_path)
        new_shipments.append(excluded_shipment)

        selected_pairs = [(i, j) for (i, j) in route_pairs if x[(i, j)].solution_value == 1]
        # print("Selected Route Pairs:", selected_pairs)
        # print("Routes Before")
        # print(shipment_routes)
        for pair in selected_pairs:
          s1 = pair[0]
          s2 = pair[1]
          R1 = R[s1]
          # print("Route 1")
          # print(R1)
          R2 = R[s2]
          # print("Route 2")
          # print(R2)
          combined_shipment, merged_path, cost = merge(R1, R2, min_path_shipment_dict_copy)
          # print("Merged Path")
          # print(merged_path)
          new_routes.append(merged_path)
          new_shipments.append(combined_shipment)
          # print("Routes After")
          # print(new_routes)
          shipment_min_path_dict_copy[combined_shipment] = merged_path
          min_path_shipment_dict_copy[tuple(merged_path)] = combined_shipment

    shipment_routes = new_routes.copy()
    shipments = new_shipments.copy()
  return shipments, shipment_routes

In [ ]:
def T_WM(shipments, K):
    shipment_min_path_dict_copy = shipment_min_path_dict.copy()
    min_path_shipment_dict_copy = min_path_shipment_dict.copy()

    shipment_routes = [shipment_min_path_dict_copy[s] for s in shipments] # make this a local copy and mutate it
    index_shipment_routes = [i for i in range(0, len(shipment_routes))]
    R = dict(zip(index_shipment_routes, shipment_routes))

    route_pairs = [(i, j) for j in range(1, len(index_shipment_routes)) for i in range(0, j)]

    subgraph = {key : [] for key in index_shipment_routes}

    for (i, j) in route_pairs:
        path_cost, twisted = marginal_cost(R[i], R[j], min_path_shipment_dict_copy)
        if twisted:
            subgraph[i].append(j)

    connected_components = find_all_connected_components(subgraph)
    # print(connected_components)
    isolated_paths = []
    isolated_shipments = []

    for c in connected_components:
        c_paths = []
        c_shipments = []
        for i in c:
            c_paths.append(R[i])
            c_shipments.append(shipments[i])
        # print(c_paths[0])
        combined_shipments, routes = WM_helper(c_shipments, c_paths, 1, shipment_min_path_dict_copy, min_path_shipment_dict_copy)
        isolated_paths.append(routes[0])
        isolated_shipments.append(combined_shipments)

    isolated_shipments = [i[0] for i in isolated_shipments]

    updated_shipment_min_path_dict = dict(zip(isolated_shipments, isolated_paths))
    updated_min_path_shipment_dict = dict(zip([tuple(path) for path in isolated_paths], isolated_shipments))

    _, routes = WM_helper(isolated_shipments, isolated_paths, K, updated_shipment_min_path_dict, updated_min_path_shipment_dict)

    return routes

To run this algorithms, the user will need to install CPLEX within the environment. This will install CPLEX Community Edition which limits the user to a maximum of 64 shipments. For the full range of tests, IBM CPLEX OPTIMIZATION Studio is required.